<a href="https://colab.research.google.com/github/ChapelFob80930/AI-Generated-Text-Detector/blob/main/AITextTest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

train_prompts = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/train_prompts.csv")
print(train_prompts.head())
print("\n")
train_essays = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/train_essays.csv")
print(train_essays.head())
print("\n")
test_essays = pd.read_csv("https://raw.githubusercontent.com/ChapelFob80930/AI-Generated-Text-Detector/refs/heads/main/test_essays.csv")
print(test_essays.head())
ai_human_dataset = pd.read_csv("/content/AI_Human.csv")
print(ai_human_dataset.head())

   prompt_id                       prompt_name  \
0          0                   Car-free cities   
1          1  Does the electoral college work?   

                                        instructions  \
0  Write an explanatory essay to inform fellow ci...   
1  Write a letter to your state senator in which ...   

                                         source_text  
0  # In German Suburb, Life Goes On Without Cars ...  
1  # What Is the Electoral College? by the Office...  


         id  prompt_id                                               text  \
0  0059830c          0  Cars. Cars have been around since they became ...   
1  005db917          0  Transportation is a large necessity in most co...   
2  008f63e3          0  "America's love affair with it's vehicles seem...   
3  00940276          0  How often do you ride in a car? Do you drive a...   
4  00c39458          0  Cars are a wonderful thing. They are perhaps o...   

   generated  
0          0  
1          0  
2    

In [2]:
import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_eng to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package averaged_perceptron_tagger_rus to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_rus.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_d


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')

def lemmatize_text(text):
  doc = nlp(" ".join(text))
  return [token.lemma_ for token in doc]

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def _remove_stopwords_(texts):
  return [text for text in texts if text.isalnum() and text not in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def preprocess_text(series):
    def process(text):
        tokens = word_tokenize(text)
        tokens = _remove_stopwords_(tokens)
        tokens = lemmatize_text(tokens)
        return " ".join(tokens)
    return series.apply(process)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
# train_essays["preprocessedText"]=preprocess_text(train_essays['text'])

In [ ]:
ai_human_dataset["preprocessedText"]=preprocess_text(ai_human_dataset['text'])

In [ ]:
# print(train_essays.head())

In [ ]:
print(ai_human_dataset.head())

In [ ]:
# from gensim.models import Word2Vec

# sentences = train_essays['preprocessedText'].apply(lambda x: x.split())

# word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# word_vector = word2vec_model.wv['car']
# print("Vector for 'car':", word_vector)

# # train_essays['word2vecVector'] = sentences.apply(
# #     lambda x: sum([word2vec_model.wv[word] for word in x if word in word2vec_model.wv]) / len(x)
# # )

# def calculate_average_word_vector(words,model,vector_size=100):
#     word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
#     if word_vectors:
#         return np.mean(word_vectors, axis=0)
#     else:
#         return np.zeros(vector_size)

# train_essays['word2vecVector'] = train_essays['preprocessedText'].apply(lambda x: calculate_average_word_vector(x.split(),word2vec_model))


In [ ]:
from gensim.models import Word2Vec

sentences = ai_human_dataset["preprocessedText"].apply(lambda x: x.split())

word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

word_vector = word2vec_model.wv['car']
print("Vector for 'car':", word_vector)

# train_essays['word2vecVector'] = sentences.apply(
#     lambda x: sum([word2vec_model.wv[word] for word in x if word in word2vec_model.wv]) / len(x)
# )

def calculate_average_word_vector(words,model,vector_size=100):
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(vector_size)

ai_human_dataset['word2vecVector'] = ai_human_dataset["preprocessedText"].apply(lambda x: calculate_average_word_vector(x.split(),word2vec_model))


In [ ]:
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(train_essays['preprocessedText'])]

# doc2vec_model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4)

# train_essays['doc2vecVector'] = train_essays.index.map(lambda x: doc2vec_model.dv[str(x)])


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_data = [TaggedDocument(words=text.split(), tags=[str(i)]) for i, text in enumerate(ai_human_dataset["preprocessedText"])]

doc2vec_model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=1, workers=4)

ai_human_dataset['doc2vecVector'] = ai_human_dataset.index.map(lambda x: doc2vec_model.dv[str(x)])

In [ ]:
print(train_essays.head())

In [ ]:
# from sklearn.model_selection import train_test_split

# y = train_essays['generated']

# train_index, test_index = train_test_split(train_essays.index, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

y = ai_human_dataset['generated']

train_index, test_index = train_test_split(ai_human_dataset.index, test_size=0.2, random_state=42)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf_vectorizer = TfidfVectorizer(max_features=1000)
# X_tfidf = tfidf_vectorizer.fit_transform(train_essays['preprocessedText'])

# X_train_tfidf, X_test_tfidf = X_tfidf[train_index], X_tfidf[test_index]
# y_train, y_test = y[train_index], y[test_index]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_tfidf = tfidf_vectorizer.fit_transform(ai_human_dataset["preprocessedText"])

X_train_tfidf, X_test_tfidf = X_tfidf[train_index], X_tfidf[test_index]
y_train, y_test = y[train_index], y[test_index]


In [ ]:
ai_human_dataset['word2vecVector'] = ai_human_dataset['word2vecVector'].apply(lambda x: np.array(x))

X_word2vec = np.stack(ai_human_dataset['word2vecVector'].values)

X_train_word2vec, X_test_word2vec = X_word2vec[train_index], X_word2vec[test_index]

print(X_train_word2vec.shape)
print(X_test_word2vec.shape)
print(X_train_word2vec)

In [ ]:
# train_essays['word2vecVector'] = train_essays['word2vecVector'].apply(lambda x: np.array(x))

# X_word2vec = np.stack(train_essays['word2vecVector'].values)

# X_train_word2vec, X_test_word2vec = X_word2vec[train_index], X_word2vec[test_index]

# print(X_train_word2vec.shape)
# print(X_test_word2vec.shape)
# print(X_train_word2vec)

In [ ]:
ai_human_dataset['doc2vecVector'] = ai_human_dataset['doc2vecVector'].apply(lambda x: np.array(x))
X_doc2vec = np.stack(ai_human_dataset['doc2vecVector'].values)
X_train_doc2vec, X_test_doc2vec = X_doc2vec[train_index], X_doc2vec[test_index]
print(X_train_doc2vec.shape)
print(X_test_doc2vec.shape)
print(X_train_doc2vec)

In [ ]:
# train_essays['doc2vecVector'] = train_essays['doc2vecVector'].apply(lambda x: np.array(x))
# X_doc2vec = np.stack(train_essays['doc2vecVector'].values)
# X_train_doc2vec, X_test_doc2vec = X_doc2vec[train_index], X_doc2vec[test_index]
# print(X_train_doc2vec.shape)
# print(X_test_doc2vec.shape)
# print(X_train_doc2vec)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def evaluate_svm_model(X_train, X_test, y_train, y_test):
    svm_model = SVC(class_weight='balanced')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }


results_tfidf = evaluate_svm_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for SVM:", results_tfidf)


results_word2vec = evaluate_svm_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for SVM:", results_word2vec)


results_doc2vec = evaluate_svm_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for SVM:", results_doc2vec)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
def evaluate_decisionTree_model(X_train, X_test, y_train, y_test):
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    y_pred = dt_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }

results_tfidf = evaluate_decisionTree_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for Decision Tree:", results_tfidf)


results_word2vec = evaluate_decisionTree_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for Decision Tree:", results_word2vec)


results_doc2vec = evaluate_decisionTree_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for Decision Tree:", results_doc2vec)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
def evaluate_randomForest_model(X_train, X_test, y_train, y_test):
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }
results_tfidf = evaluate_randomForest_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for Random Forest:", results_tfidf)


results_word2vec = evaluate_randomForest_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for Random Forest:", results_word2vec)


results_doc2vec = evaluate_randomForest_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for Random Forest:", results_doc2vec)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
def evaluate_knn_model(X_train, X_test, y_train, y_test):
    knn_model = KNeighborsClassifier()
    knn_model.fit(X_train, y_train)
    y_pred = knn_model.predict(X_test)
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred,),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred)
    }
results_tfidf = evaluate_knn_model(X_train_tfidf, X_test_tfidf, y_train, y_test)
print("TF-IDF Results for KNN:", results_tfidf)


results_word2vec = evaluate_knn_model(X_train_word2vec, X_test_word2vec, y_train, y_test)
print("Word2Vec Results for KNN:", results_word2vec)


results_doc2vec = evaluate_knn_model(X_train_doc2vec, X_test_doc2vec, y_train, y_test)
print("Doc2Vec Results for KNN:", results_doc2vec)

#when u put in the correct dataset use error rate (1-accuracy) vs k values to get optimal k value at elbow point
